<a href="https://colab.research.google.com/github/paresh-nayak04/Plagiarism-Detector/blob/main/PLAGAIRISM_DETECTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install streamlit
!pip install docx2txt
!pip install PyPDF2
import streamlit as st
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import tokenize
from bs4 import BeautifulSoup
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import io
import docx2txt
from PyPDF2 import PdfReader
import plotly.express as px

def get_sentences(text):
    sentences = tokenize.sent_tokenize(text)
    return sentences

def get_url(sentence):
    base_url = 'https://www.google.com/search?q='
    query = sentence
    query = query.replace(' ', '+')
    url = base_url + query
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    divs = soup.find_all('div', class_='yuRUbf')
    urls = []
    for div in divs:
        a = div.find('a')
        urls.append(a['href'])
    if len(urls) == 0:
        return None
    elif "youtube" in urls[0]:
        return None
    else:
        return urls[0]

def read_text_file(file):
    content = ""
    with io.open(file.name, 'r', encoding='utf-8') as f:
        content = f.read()
    return content

def read_docx_file(file):
    text = docx2txt.process(file)
    return text

def read_pdf_file(file):
    text = ""
    pdf_reader = PdfReader(file)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def get_text_from_file(uploaded_file):
    content = ""
    if uploaded_file is not None:
        if uploaded_file.type == "text/plain":
            content = read_text_file(uploaded_file)
        elif uploaded_file.type == "application/pdf":
            content = read_pdf_file(uploaded_file)
        elif uploaded_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
            content = read_docx_file(uploaded_file)
    return content

def get_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text

def get_similarity(text1, text2):
    text_list = [text1, text2]
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(text_list)
    similarity = cosine_similarity(count_matrix)[0][1]
    return similarity

def get_similarity_list(texts, filenames=None):
    similarity_list = []
    if filenames is None:
        filenames = [f"File {i+1}" for i in range(len(texts))]
    for i in range(len(texts)):
        for j in range(i+1, len(texts)):
            similarity = get_similarity(texts[i], texts[j])
            similarity_list.append((filenames[i], filenames[j], similarity))
    return similarity_list
def get_similarity_list2(text, url_list):
    similarity_list = []
    for url in url_list:
        text2 = get_text(url)
        similarity = get_similarity(text, text2)
        similarity_list.append(similarity)
    return similarity_list

def plot_scatter(df):
    fig = px.scatter(df, x='File 1', y='File 2', color='Similarity', title='Similarity Scatter Plot')
    st.plotly_chart(fig, use_container_width=True)

def plot_line(df):
    fig = px.line(df, x='File 1', y='File 2', color='Similarity', title='Similarity Line Chart')
    st.plotly_chart(fig, use_container_width=True)

def plot_bar(df):
    fig = px.bar(df, x='File 1', y='Similarity', color='File 2', title='Similarity Bar Chart')
    st.plotly_chart(fig, use_container_width=True)

def plot_pie(df):
    fig = px.pie(df, values='Similarity', names='File 1', title='Similarity Pie Chart')
    st.plotly_chart(fig, use_container_width=True)

def plot_box(df):
    fig = px.box(df, x='File 1', y='Similarity', title='Similarity Box Plot')
    st.plotly_chart(fig, use_container_width=True)

def plot_histogram(df):
    fig = px.histogram(df, x='Similarity', title='Similarity Histogram')
    st.plotly_chart(fig, use_container_width=True)

def plot_3d_scatter(df):
    fig = px.scatter_3d(df, x='File 1', y='File 2', z='Similarity', color='Similarity',
                        title='Similarity 3D Scatter Plot')
    st.plotly_chart(fig, use_container_width=True)

def plot_violin(df):
    fig = px.violin(df, y='Similarity', x='File 1', title='Similarity Violin Plot')
    st.plotly_chart(fig, use_container_width=True)


st.set_page_config(page_title='Plagiarism Detection')
st.title('Plagiarism Detector')

st.write("""
### Enter the text or upload a file to check for plagiarism or find similarities between files
""")
option = st.radio(
    "Select input option:",
    ('Enter text', 'Upload file', 'Find similarities between files')
)

if option == 'Enter text':
    text = st.text_area("Enter text here", height=200)
    uploaded_files = []
elif option == 'Upload file':
    uploaded_file = st.file_uploader("Upload file (.docx, .pdf, .txt)", type=["docx", "pdf", "txt"])
    if uploaded_file is not None:
        text = get_text_from_file(uploaded_file)
        uploaded_files = [uploaded_file]
    else:
        text = ""
        uploaded_files = []
else:
    uploaded_files = st.file_uploader("Upload multiple files (.docx, .pdf, .txt)", type=["docx", "pdf", "txt"], accept_multiple_files=True)
    texts = []
    filenames = []
    for uploaded_file in uploaded_files:
        if uploaded_file is not None:
            text = get_text_from_file(uploaded_file)
            texts.append(text)
            filenames.append(uploaded_file.name)
    text = " ".join(texts)

if st.button('Check for plagiarism or find similarities'):
    st.write("""
    ### Checking for plagiarism or finding similarities...
    """)
    if not text:
        st.write("""
        ### No text found for plagiarism check or finding similarities.
        """)
        st.stop()

    if option == 'Find similarities between files':
        similarities = get_similarity_list(texts, filenames)
        df = pd.DataFrame(similarities, columns=['File 1', 'File 2', 'Similarity'])
        df = df.sort_values(by=['Similarity'], ascending=False)
        # Plotting interactive graphs
        plot_scatter(df)
        plot_line(df)
        plot_bar(df)
        plot_pie(df)
        plot_box(df)
        plot_histogram(df)
        plot_3d_scatter(df)
        plot_violin(df)
    else:
        sentences = get_sentences(text)
        url = []
        for sentence in sentences:
            url.append(get_url(sentence))

        if None in url:
            st.write("""
            ### No plagiarism detected!
            """)
            st.stop()

        similarity_list = get_similarity_list2(text, url)
        df = pd.DataFrame({'Sentence': sentences, 'URL': url, 'Similarity': similarity_list})
        df = df.sort_values(by=['Similarity'], ascending=True)

    df = df.reset_index(drop=True)

    # Make URLs clickable in the DataFrame
    if 'URL' in df.columns:
        df['URL'] = df['URL'].apply(lambda x: '<a href="{}">{}</a>'.format(x, x) if x else '')

    # Center align URL column header
    df_html = df.to_html(escape=False)
    if 'URL' in df.columns:
        df_html = df_html.replace('<th>URL</th>', '<th style="text-align: center;">URL</th>')
    st.write(df_html, unsafe_allow_html=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-11-10 14:18:41.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 14:18:41.597 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 14:18:41.597 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 14:18:41.599 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 14:18:41.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 14:18:41.601 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 14:18:41.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11

To run your Streamlit application, you need to:

1. Save the code from the previous cell into a Python file (e.g., `app.py`).
2. Run the Streamlit application using the `streamlit run` command, along with `npx localtunnel` to expose the app publicly.

Here's how you can do it:


In [25]:
%%writefile app.py
# This cell writes the content of the previous code cell to a file named 'app.py'
# Please ensure the previous cell contains the complete Streamlit application code

import streamlit as st
import pandas as pd
import nltk
nltk.download('punkt') # Ensure 'punkt' is downloaded
nltk.download('punkt_tab') # Download 'punkt_tab' for the tokenizer
from nltk import tokenize
from bs4 import BeautifulSoup
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import io
import docx2txt
from PyPDF2 import PdfReader
import plotly.express as px

def get_sentences(text):
    sentences = tokenize.sent_tokenize(text)
    return sentences

def get_url(sentence):
    base_url = 'https://www.google.com/search?q='
    query = sentence
    query = query.replace(' ', '+')
    url = base_url + query
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    divs = soup.find_all('div', class_='yuRUbf')
    urls = []
    for div in divs:
        a = div.find('a')
        urls.append(a['href'])
    if len(urls) == 0:
        return None
    elif "youtube" in urls[0]:
        return None
    else:
        return urls[0]

def read_text_file(file):
    # Streamlit's uploaded file is a BytesIO-like object
    content = file.getvalue().decode('utf-8')
    return content

def read_docx_file(file):
    # docx2txt can process file-like objects directly
    text = docx2txt.process(file)
    return text

def read_pdf_file(file):
    text = ""
    # PyPDF2 PdfReader can process file-like objects directly
    pdf_reader = PdfReader(file)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def get_text_from_file(uploaded_file):
    content = ""
    if uploaded_file is not None:
        if uploaded_file.type == "text/plain":
            content = read_text_file(uploaded_file)
        elif uploaded_file.type == "application/pdf":
            content = read_pdf_file(uploaded_file)
        elif uploaded_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
            content = read_docx_file(uploaded_file)
    return content

def get_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text

def get_similarity(text1, text2):
    text_list = [text1, text2]
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(text_list)
    similarity = cosine_similarity(count_matrix)[0][1]
    return similarity

def get_similarity_list(texts, filenames=None):
    similarity_list = []
    if filenames is None:
        filenames = [f"File {i+1}" for i in range(len(texts))]
    for i in range(len(texts)):
        for j in range(i+1, len(texts)):
            similarity = get_similarity(texts[i], texts[j])
            similarity_list.append((filenames[i], filenames[j], similarity))
    return similarity_list
def get_similarity_list2(text, url_list):
    similarity_list = []
    for url in url_list:
        text2 = get_text(url)
        similarity = get_similarity(text, text2)
        similarity_list.append(similarity)
    return similarity_list

def plot_scatter(df):
    fig = px.scatter(df, x='File 1', y='File 2', color='Similarity', title='Similarity Scatter Plot')
    st.plotly_chart(fig, use_container_width=True)

def plot_line(df):
    fig = px.line(df, x='File 1', y='File 2', color='Similarity', title='Similarity Line Chart')
    st.plotly_chart(fig, use_container_width=True)

def plot_bar(df):
    fig = px.bar(df, x='File 1', y='Similarity', color='File 2', title='Similarity Bar Chart')
    st.plotly_chart(fig, use_container_width=True)

def plot_pie(df):
    fig = px.pie(df, values='Similarity', names='File 1', title='Similarity Pie Chart')
    st.plotly_chart(fig, use_container_width=True)

def plot_box(df):
    fig = px.box(df, x='File 1', y='Similarity', title='Similarity Box Plot')
    st.plotly_chart(fig, use_container_width=True)

def plot_histogram(df):
    fig = px.histogram(df, x='Similarity', title='Similarity Histogram')
    st.plotly_chart(fig, use_container_width=True)

def plot_3d_scatter(df):
    fig = px.scatter_3d(df, x='File 1', y='File 2', z='Similarity', color='Similarity',
                        title='Similarity 3D Scatter Plot')
    st.plotly_chart(fig, use_container_width=True)

def plot_violin(df):
    fig = px.violin(df, y='Similarity', x='File 1', title='Similarity Violin Plot')
    st.plotly_chart(fig, use_container_width=True)


st.set_page_config(page_title='Plagiarism Detection')
st.title('Plagiarism Detector')

st.write("""
### Enter the text or upload a file to check for plagiarism or find similarities between files
""")
option = st.radio(
    "Select input option:",
    ('Enter text', 'Upload file', 'Find similarities between files')
)

if option == 'Enter text':
    text = st.text_area("Enter text here", height=200)
    uploaded_files = []
elif option == 'Upload file':
    uploaded_file = st.file_uploader("Upload file (.docx, .pdf, .txt)", type=["docx", "pdf", "txt"])
    if uploaded_file is not None:
        text = get_text_from_file(uploaded_file)
        uploaded_files = [uploaded_file]
    else:
        text = ""
        uploaded_files = []
else:
    uploaded_files = st.file_uploader("Upload multiple files (.docx, .pdf, .txt)", type=["docx", "pdf", "txt"], accept_multiple_files=True)
    texts = []
    filenames = []
    for uploaded_file in uploaded_files:
        if uploaded_file is not None:
            text = get_text_from_file(uploaded_file)
            texts.append(text)
            filenames.append(uploaded_file.name)
    text = " ".join(texts)

if st.button('Check for plagiarism or find similarities'):
    st.write("""
    ### Checking for plagiarism or finding similarities...
    """)
    if not text:
        st.write("""
        ### No text found for plagiarism check or finding similarities.
        """)
        st.stop()

    if option == 'Find similarities between files':
        similarities = get_similarity_list(texts, filenames)
        df = pd.DataFrame(similarities, columns=['File 1', 'File 2', 'Similarity'])
        df = df.sort_values(by=['Similarity'], ascending=False)
        # Plotting interactive graphs
        plot_scatter(df)
        plot_line(df)
        plot_bar(df)
        plot_pie(df)
        plot_box(df)
        plot_histogram(df)
        plot_3d_scatter(df)
        plot_violin(df)
    else: # Plagiarism check for single input text/file
        sentences = get_sentences(text)
        if not sentences:
            st.write("### No sentences found in the provided text for plagiarism check.")
            st.stop()

        results_data = []
        for sentence in sentences:
            online_url = get_url(sentence) # Get the most relevant URL for the sentence
            if online_url:
                try:
                    # Fetch content from the URL
                    online_text = get_text(online_url)
                    # Calculate similarity between the input sentence and the online content
                    similarity_score = get_similarity(sentence, online_text)
                    results_data.append({'Sentence': sentence, 'URL': online_url, 'Similarity': similarity_score})
                except requests.exceptions.RequestException:
                    # Handle cases where fetching the URL content fails (e.g., connection error, 404)
                    results_data.append({'Sentence': sentence, 'URL': online_url + " (Error fetching content)", 'Similarity': 0.0})
                except Exception as e:
                    # Catch any other unexpected errors during processing
                    print(f"An unexpected error occurred for sentence: '{sentence}' and URL: {online_url} - {e}")
                    results_data.append({'Sentence': sentence, 'URL': online_url + " (Processing error)", 'Similarity': 0.0})
            else:
                # If no relevant online URL was found for the sentence
                results_data.append({'Sentence': sentence, 'URL': 'No direct online match found', 'Similarity': 0.0})

        if not results_data:
            st.write("### Could not process any sentences for plagiarism check.")
            st.stop()

        df = pd.DataFrame(results_data)
        # Sort by similarity in descending order to show highest matches first
        df = df.sort_values(by=['Similarity'], ascending=False)

    df = df.reset_index(drop=True)

    # Make URLs clickable in the DataFrame
    if 'URL' in df.columns:
        df['URL'] = df['URL'].apply(lambda x: '<a href="{}">{}</a>'.format(x, x) if x else '')

    # Center align URL column header
    df_html = df.to_html(escape=False)
    if 'URL' in df.columns:
        df_html = df_html.replace('<th>URL</th>', '<th style="text-align: center;">URL</th>')
    st.write(df_html, unsafe_allow_html=True)

Overwriting app.py


In [10]:
!pip install streamlit pyngrok docx2txt PyPDF2 plotly scikit-learn beautifulsoup4 nltk

In [22]:
from pyngrok import ngrok

# Kill any existing tunnels (clean start)
ngrok.kill()

# !! IMPORTANT: Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken !!
# You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("35I78QR6zTiSH9DcWGGu65enOh1_G1NW859WXgp2iPPwQvgd")

# Start a new tunnel on port 8501
public_url = ngrok.connect(8501)
print(f"🌍 Your Streamlit app will be live here:\n{public_url}")

🌍 Your Streamlit app will be live here:
NgrokTunnel: "https://crosby-neighborly-charity.ngrok-free.dev" -> "http://localhost:8501"


### Debugging `get_url` function
Let's test the `get_url` function with a sample sentence from Wikipedia to see if it correctly extracts a URL. The most common reason for this issue is that Google's HTML structure for search results has changed, making the `yuRUbf` class selector outdated.

In [24]:
sample_wikipedia_sentence = "Wikipedia is a multilingual free online encyclopedia written and maintained by a community of volunteers through open collaboration and a wiki-based editing system."

# Re-define get_url (copying from app.py to ensure we're testing the exact logic)
def get_url(sentence):
    base_url = 'https://www.google.com/search?q='
    query = sentence
    query = query.replace(' ', '+')
    url = base_url + query
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    print(f"HTTP Status Code: {res.status_code}")
    print(f"First 500 chars of response: {res.text[:500]}")

    urls = []

    # Try to find common Google search result links
    # Look for a tags that are direct descendants of div elements with specific roles or data attributes
    # Or links that start with /url?q=
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        if href.startswith('/url?q=') and not href.startswith('/url?q=/search'):
            # Extract the clean URL from Google's redirect link
            clean_url = href.split('/url?q=')[1].split('&sa=U')[0]
            urls.append(clean_url)
        elif 'wikipedia.org' in href and not href.startswith('/') and 'google.com' not in href:
            # A fallback for direct Wikipedia links if the /url?q= pattern fails or is not primary
            urls.append(href)
        # For debugging, print all found hrefs
        print(f"Found href: {href}")

    if not urls:
        # Fallback to broader search, looking for links within common result blocks
        # Often, main results are within div tags with specific classes or roles
        for div in soup.find_all('div', class_=['g', 'rc', 'yuRUbf', 'tF2CMy']):
            a = div.find('a', href=True)
            if a and not a['href'].startswith('/search') and 'google.com' not in a['href']:
                urls.append(a['href'])

    # Filter out YouTube links and return the first non-YouTube URL
    filtered_urls = [u for u in urls if "youtube.com" not in u and "support.google.com" not in u and u.startswith('http')]

    if len(filtered_urls) == 0:
        return None
    else:
        return filtered_urls[0]

found_url = get_url(sample_wikipedia_sentence)
print(f"Final Found URL: {found_url}")

HTTP Status Code: 200
First 500 chars of response: <!DOCTYPE html><html lang="en"><head><title>Google Search</title><style>body{background-color:var(--xhUGwc)}</style><script nonce="7XzTKZpUti011DV2uqnasA">window.google = window.google || {};window.google.c = window.google.c || {cap:0};</script></head><body><noscript><style>table,div,span,p{display:none}</style><meta content="0;url=/httpservice/retry/enablejs?sei=nf4RabK-KuPdwN4PxITd2Aw" http-equiv="refresh"><div style="display:block">Please click <a href="/httpservice/retry/enablejs?sei=nf4RabK
Found href: /httpservice/retry/enablejs?sei=nf4RabK-KuPdwN4PxITd2Aw
Found href: /search?q=Wikipedia+is+a+multilingual+free+online+encyclopedia+written+and+maintained+by+a+community+of+volunteers+through+open+collaboration+and+a+wiki-based+editing+system.&sca_esv=3f626f5782a50780&emsg=SG_REL&sei=nf4RabK-KuPdwN4PxITd2Aw
Found href: https://support.google.com/websearch
Final Found URL: None


In [17]:
!streamlit run app.py --server.port 8501 &>/dev/null&